In [1]:
import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#---------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Plot_Box_sns
import GrubbsTest

# NOTES

instantaneous is old format
<br> Horizontal format with one row for each meter(?)

usage_instantaneous is new format
<br> Tall format with multiple rows for each meter

For each, the database to use is inst_msr_consume (I believe)


Also, look into meter_events.end_device_event

In [2]:
my_meter_premise_info = pd.read_csv(r'C:\Users\s346557\Documents\my_meter_premise_info.csv')
my_meter_premise_info = clm.remove_prepend_from_columns_in_df(my_meter_premise_info)
my_meter_premise_info_dict = my_meter_premise_info.squeeze().to_dict()

# Comparing instantaneous vs usage_instantaneous

In [3]:
conn_aws = Utilities.get_athena_prod_aws_connection()

In [4]:
# cols_of_interest_usage_inst = [
#     'read_type',
#     'serialnumber',
#     'aep_premise_nb',
#     'timezoneoffset',
#     'aep_readtime',
#     'aep_readtime_utc',
#     'measurement_type',
#     'measurement_value',
#     'longitude',
#     'latitude',
#     'aep_opco',
#     'aep_read_dt'
# ]

# cols_of_interest_inst = [
#     'meter_id', 
#     'read_time', 
#     'read_date', 
#     'msr_value', 
#     'msr_type'
# ]

cols_of_interest_usage_inst = [
    '*'
]

cols_of_interest_inst = [
    '*'
]

In [5]:
date_range = ['2021-10-12', '2021-11-13']

In [6]:
sql_usg_inst = (
"""
SELECT {}
FROM usage_instantaneous.inst_msr_consume
WHERE aep_opco = 'oh' 
AND aep_read_dt BETWEEN '{}' AND '{}'
AND aep_premise_nb = '{}'
AND serialnumber = '{}'
"""
).format(','.join(cols_of_interest_usage_inst), 
         date_range[0], 
         date_range[1], 
         my_meter_premise_info_dict['prem_nb'], 
         my_meter_premise_info_dict['mfr_devc_ser_nbr'])

In [7]:
print(sql_usg_inst)


SELECT *
FROM usage_instantaneous.inst_msr_consume
WHERE aep_opco = 'oh' 
AND aep_read_dt BETWEEN '2021-10-12' AND '2021-11-13'
AND aep_premise_nb = '104752350'
AND serialnumber = '879838382'



In [8]:
sql_inst = (
"""
SELECT {}
FROM instantaneous.inst_msr_consume
WHERE opco_cd = 'oh' 
AND read_date BETWEEN '{}' AND '{}'
AND prem_nb = '{}'
AND meter_id = '{}'
"""
).format(','.join(cols_of_interest_inst), 
         date_range[0], 
         date_range[1],  
         my_meter_premise_info_dict['prem_nb'], 
         my_meter_premise_info_dict['mfr_devc_ser_nbr'])

In [9]:
print(sql_inst)


SELECT *
FROM instantaneous.inst_msr_consume
WHERE opco_cd = 'oh' 
AND read_date BETWEEN '2021-10-12' AND '2021-11-13'
AND prem_nb = '104752350'
AND meter_id = '879838382'



In [10]:
print('df_usg_inst')
print('-'*10)
print(sql_usg_inst)
print('*'*50)
print('df_inst')
print('-'*10)
print(sql_inst)

df_usg_inst
----------

SELECT *
FROM usage_instantaneous.inst_msr_consume
WHERE aep_opco = 'oh' 
AND aep_read_dt BETWEEN '2021-10-12' AND '2021-11-13'
AND aep_premise_nb = '104752350'
AND serialnumber = '879838382'

**************************************************
df_inst
----------

SELECT *
FROM instantaneous.inst_msr_consume
WHERE opco_cd = 'oh' 
AND read_date BETWEEN '2021-10-12' AND '2021-11-13'
AND prem_nb = '104752350'
AND meter_id = '879838382'



In [ ]:
df_usg_inst_OG = pd.read_sql(sql_usg_inst, conn_aws)
df_usg_inst_OG = clm.remove_table_aliases(df_usg_inst_OG)

In [ ]:
df_inst_OG = pd.read_sql(sql_inst, conn_aws)
df_inst_OG = clm.remove_table_aliases(df_inst_OG)

In [ ]:
df_usg_inst_OG.head()

In [ ]:
df_inst_OG.head()

# -----

In [ ]:
df_usg_inst = df_usg_inst_OG.copy()
df_inst = df_inst_OG.copy()

In [ ]:
print(f'df_usg_inst.shape = {df_usg_inst.shape}')
print(f'df_inst.shape     = {df_inst.shape}')
print('\n'+'*'*50)
# df_usg_inst has many more measurement types than df_inst
print("\ndf_usg_inst['measurement_type'].unique()\n", '-'*25+'\n', df_usg_inst['measurement_type'].unique())

print("\ndf_inst['msr_type'].unique()\n", '-'*25+'\n', df_inst['msr_type'].unique())

In [ ]:
cols_to_comp = ['serialnumber', 'aep_readtime', 'measurement_value', 'measurement_type']
msr_type_translation = {
    'InstAMPsCalc':'instantaneous_ia', 
    'InstkW':'instantaneous_kw', 
    'PowerFactorMeter':'power_factor_phase_a',
    'TempF':'temperature_f', 
    'VoltageA':'voltage_phase_a'
}
#--------------------------------------------
# Build aep_readtime in df_inst by combining the read_date and read_time fields
df_inst['aep_readtime'] = pd.to_datetime(df_inst['read_date']+' '+df_inst['read_time'])
#-----
# Rename columns in df_inst to match those in df_usg_inst
df_inst = df_inst.rename(columns={'meter_id':'serialnumber', 
                                  'msr_value':'measurement_value', 
                                  'msr_type':'measurement_type'})
# Change the msr_type values in df_inst to match those in df_usg_inst
for inst_msr_type, usg_inst_msr_type in msr_type_translation.items():
    df_inst.loc[df_inst['measurement_type']==inst_msr_type, 'measurement_type']=usg_inst_msr_type
#-----
df_inst = Utilities_df.convert_col_types(
    df=df_inst, 
    cols_and_types_dict={'measurement_value':float}, 
    to_numeric_errors='coerce', 
    inplace=True
)
#-----
df_inst = Utilities_df.move_cols_to_front(df_inst, cols_to_comp) 

In [ ]:
# Select only the measurement types contained in df_inst
# The following line is what takes the number of entries in df_usg_inst from 
# 1017 to 990
df_usg_inst=df_usg_inst[df_usg_inst['measurement_type'].isin(msr_type_translation.values())]
print(f'df_usg_inst.shape = {df_usg_inst.shape}')
print(f'df_inst.shape     = {df_inst.shape}')
#-----
df_usg_inst['aep_readtime'] = pd.to_datetime(df_usg_inst['aep_readtime'])
df_usg_inst = Utilities_df.convert_col_types(
    df=df_usg_inst, 
    cols_and_types_dict={'measurement_value':float}, 
    to_numeric_errors='coerce', 
    inplace=True
)
df_usg_inst = Utilities_df.move_cols_to_front(df_usg_inst, cols_to_comp)

In [ ]:
# What is load_ts, and why does it cause so many duplicate values?
#   i.e., instead of a single row, there are multiple rows which are exactly
#     the same except for load_ts
df_inst_dupl = df_inst[df_inst.duplicated(subset=[x for x in df_inst.columns if x!='load_ts'], keep=False)]
print(f"df_inst_dupl.shape = {df_inst_dupl.shape}")
print(f"df_inst_dupl['load_ts'].nunique() = {df_inst_dupl['load_ts'].nunique()}")

df_inst_dupl_gpd = df_inst_dupl.groupby([x for x in df_inst.columns if x!='load_ts'])

In [ ]:
get_group_idx = 0 
df_inst_dupl_gpd.get_group(list(df_inst_dupl_gpd.groups.keys())[get_group_idx])

In [ ]:
get_group_idx = 1
df_inst_dupl_gpd.get_group(list(df_inst_dupl_gpd.groups.keys())[get_group_idx])

In [ ]:
# Remove duplicate entries which are exactly the same except for the load_ts field
# This takes the size of df_inst form 3770 to 990
df_inst = df_inst.drop_duplicates(subset=[x for x in df_inst.columns if x!='load_ts'])
df_inst = Utilities_df.move_cols_to_front(df_inst, cols_to_comp)

print(f'df_usg_inst.shape = {df_usg_inst.shape}')
print(f'df_inst.shape     = {df_inst.shape}')

In [ ]:
sort_by = ['aep_readtime', 'measurement_type']
df_usg_inst = df_usg_inst.sort_values(by=sort_by, ignore_index=True)
df_inst = df_inst.sort_values(by=sort_by, ignore_index=True)

In [ ]:
# Now, after both being reduced and comparing the shared columns,
# df_usg_inst equals df_inst.
# ALTHOUGH, they do not match EXACTLY, but are different by e.g. rounding
print(f'df_usg_inst.equals(df_inst): {df_usg_inst.equals(df_inst)}')

In [ ]:
diffs = Utilities_df.get_dfs_diff(df_usg_inst[cols_to_comp], df_inst[cols_to_comp])
diffs

In [ ]:
diffs = Utilities_df.get_dfs_diff_approx_ok(df_usg_inst, df_inst, 
                                            cols_to_compare=cols_to_comp, 
                                            return_df_only=True)
diffs